In [8]:
# import statements
import numpy as np
from path import Planner

In [9]:
maps = {} # dictionary of maps, a map is a dictionary of coordinates and their corresponding values - one map for each layer

# instantiate map
def instantiate_map():
    map = {}
    for i in np.arange(0, 379.4, 0.1):
        for j in np.arange(0, 189.7, 0.1):
            map[(round(i, 1), round(j, 1))] = None
    return map

In [ ]:
# linear interpolation
# returns a list of points between two points

def get_points(x1, x2, y1, y2, step_size = 0.1):
    if x1 == x2:
        y_points = np.round(np.append(np.arange(y1, y2, step_size, dtype='float64'), y2), 1)
        x_points = np.round(np.full((np.size(y_points),), x1), 1)
        x_points = np.round(np.full((np.size(y_points),), x1), 1)

        assert np.size(x_points) == np.size(y_points)
        return x_points, y_points
    
    x_points = np.round(np.append(np.arange(x1, x2, step_size, dtype='float64'), x2), 1)
    y_points = np.zeros(np.size(x_points), dtype='float64')
    
    for i in range(np.size(x_points)):
        y_points[i] = round(((y2-y1)/(x2-x1))*(x_points[i]-x1) + y1, 1)
    
    assert np.size(x_points) == np.size(y_points)
    return x_points, y_points

In [11]:
def update_maps():
    path = Planner('sample_gcode/test_big_box.gcode')
    x_start = y_start = None # start point of the path set to None
    
    for coord in path.generate():
        x, y, z, command = coord
        
        # round coordinates to 1 decimal place
        x = round(x, 1)
        y = round(y, 1)
        
        # move coordinates to the left from (0, 0)
        x = x - 5.2
        y = y - 5.2
        
        if z not in maps:
            # create a new map for this z value
            maps[z] = instantiate_map()
            x_start = y_start = None
            
        # for the first point in the path
        if x_start is None and y_start is None:
            x_start = x
            y_start = y
        
        # linear interpolation of points between the start and end points
        x_points, y_points = get_points(x_start, x, y_start, y)
        
        # update the start point
        x_start = x
        y_start = y
        
        # update the map for both arms
        for x_point, y_point in zip(x_points, y_points):
            x_point = round(x_point, 1)
            y_point = round(y_point, 1)
            x_point_2 = round(x_point + 189.7, 1)
            maps[z][(x_point, y_point)] = ("arm_1", command)
            maps[z][(x_point_2, y_point)] = ("arm_2", command)

In [12]:
# clear all maps
def clear_maps():
    for map in maps:
        for key in map:
            map[key] = None

In [13]:
# clear all maps and update them
clear_maps()
update_maps()

In [25]:
# save maps to file
import pickle

count = 0
for map in maps:
    # write dictionary to pkl file
    with open('maps/map_' + str(count) + '.pkl', 'wb') as fp:
        pickle.dump(maps[map], fp)
        print(f"dictionary {count} saved successfully to file")
    count+=1

dictionary 0 saved successfully to file
dictionary 1 saved successfully to file
dictionary 2 saved successfully to file
dictionary 3 saved successfully to file
dictionary 4 saved successfully to file
dictionary 5 saved successfully to file
dictionary 6 saved successfully to file
dictionary 7 saved successfully to file
dictionary 8 saved successfully to file
dictionary 9 saved successfully to file
dictionary 10 saved successfully to file
dictionary 11 saved successfully to file
dictionary 12 saved successfully to file
dictionary 13 saved successfully to file
dictionary 14 saved successfully to file
dictionary 15 saved successfully to file
dictionary 16 saved successfully to file
dictionary 17 saved successfully to file
dictionary 18 saved successfully to file


In [22]:
print(maps[0.3][(379.3, 189.6)])
print(maps[0.3][(132.1, 4.2)])

print(maps[3.0][(379.3, 189.6)])
print(maps[3.0][(132.1, 4.2)])

('arm_2', 'G1')
('arm_1', 'G1')
('arm_2', 'G1')
('arm_1', 'G1')


In [ ]:
# extract maps from file
import pickle

maps = {}
count = 0

while True:
    try:
        # read dictionary from pkl file
        with open('maps/map_' + str(count) + '.pkl', 'rb') as fp:
            maps[count] = pickle.load(fp)
            print(f"dictionary {count} loaded successfully from file")
        count += 1
    except FileNotFoundError:
        break